In [3]:
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from csv import writer
from sklearn.feature_extraction.text import TfidfTransformer
from textblob import TextBlob
import re
import nltk
nltk.download('punkt')
finwiz_url = 'https://finviz.com/quote.ashx?t='


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
def append_list_as_row(file_name2, list_of_elem):
    # Open file in append mode
    with open(file_name2, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [22]:
news_tables = {}
tickers = ['AMZN']

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url, headers={'user-agent': 'my-app/0.0.1'})
    response = urlopen(req)
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    # Add the table to our dictionary
    news_tables[ticker] = news_table

In [23]:
parsed_news = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text()
        # splite text in the td tag into a list
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]

        # else load 'date' as the 1st element and 'time' as the second
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'
        ticker = file_name.split('_')[0]

        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, text])


In [24]:


columns = ['ticker', 'date', 'time', 'headline']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

# Web scraping taken from TowardsDataScience from Damien Boh
'Damian Boh,May 6 (2020),Sentiment Analysis of Stocks from Financial News using Python,[Source code],https://towardsdatascience.com/sentiment-analysis-of-stocks-from-financial-news-using-python-82ebdcefb638'

'Damian Boh,May 6 (2020),Sentiment Analysis of Stocks from Financial News using Python,[Source code],https://towardsdatascience.com/sentiment-analysis-of-stocks-from-financial-news-using-python-82ebdcefb638'

In [41]:

appendtext = []
for j in range(len(parsed_and_scored_news['headline'])):
  appendtext.append(parsed_and_scored_news['headline'][j])
  

In [66]:
for i in range(len(appendtext)):
  append_list_as_row('Newsfinal.csv',appendtext[i])

In [71]:
df8 = pd.read_csv('Newsfinal.csv')

In [72]:
df8['polarity'] = df8.apply(lambda x: TextBlob(x['sentence']).sentiment.polarity, axis=1)
df8['subjectivity'] = df8.apply(lambda x: TextBlob(x['sentence']).sentiment.subjectivity, axis=1)

In [76]:
df8["Ticker"] = "AMZN"
df8.to_csv("Newsfinal.csv", index=False)

In [77]:
df8

,sentence,polarity,subjectivity,Ticker
0,Virus worries and FOMO drive options bets on s...,0.00,0.00,AMZN
1,4 Farm and Construction Machinery Companies Wi...,0.08,0.27,AMZN
2,US STOCKS-Wall Street surges led by energy and...,0.00,0.00,AMZN
3,Amazon (AMZN) Stock Sinks As Market Gains: Wha...,-0.10,0.00,AMZN
4,US STOCKS-Wall Street surges led by energy and...,0.00,0.00,AMZN
...,...,...,...,...
95,3 Top Entertainment Stocks to Buy Now,0.50,0.50,AMZN
96,The Week In Cannabis: Stocks In Green FDAs CB...,-0.20,0.30,AMZN
97,Better Buy: Shopify vs. MercadoLibre,0.50,0.50,AMZN
98,2 Stock Bubbles That Could Pop in the Next Ma...,0.00,0.00,AMZN


In [78]:
parsed_and_scored_news 

,ticker,date,time,headline
0,AMZN,Jul-14-20,10:16PM,Virus worries and FOMO drive options bets on s...
1,AMZN,Jul-14-20,07:24PM,4 Farm and Construction Machinery Companies Wi...
2,AMZN,Jul-14-20,06:20PM,"US STOCKS-Wall Street surges, led by energy an..."
3,AMZN,Jul-14-20,05:45PM,Amazon (AMZN) Stock Sinks As Market Gains: Wha...
4,AMZN,Jul-14-20,04:52PM,"US STOCKS-Wall Street surges, led by energy an..."
...,...,...,...,...
95,AMZN,Jul-11-20,12:10PM,3 Top Entertainment Stocks to Buy Now
96,AMZN,Jul-11-20,12:00PM,"The Week In Cannabis: Stocks In Green, FDA's C..."
97,AMZN,Jul-11-20,10:30AM,Better Buy: Shopify vs. MercadoLibre
98,AMZN,Jul-11-20,10:11AM,2 Stock Bubbles That Could Pop in the Next Mar...


In [48]:
parsed_and_scored_news


data_test = parsed_and_scored_news[parsed_and_scored_news['date'] > 'Jul-13-20'].copy()


headlines = data_test['headline']

In [50]:
df = pd.read_csv('stock_datafinal.csv')


In [51]:
positive = df[df['Sentiment']==1]
Negative = df[df['Sentiment']==0]
positive.shape

(3341, 2)

In [52]:
Negative.shape

(1634, 2)

In [53]:
positive = positive.sample(Negative.shape[0])

In [54]:
positive.shape

(1634, 2)

In [55]:
data = positive.append(Negative,ignore_index=True)
data.tail()

,Text,Sentiment
3263,"JOSB 50% off on almost everything. Funny, they...",0
3264,DE considering buying some put options...,0
3265,Happy Easter! SHOT Setups: CS ISG CF AN NKD OW...,0
3266,CAT may try to jump on the boat with some Put ...,0
3267,"If AAP goes to over 451, I will go short. Stil...",0


In [56]:
X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Sentiment'], test_size=.1, random_state=0,
                                                    shuffle=True,stratify=data['Sentiment'] )

In [57]:
clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', SVC(C=10000, gamma='auto'))])

In [58]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=10000, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_fu

In [59]:
y_pred = clf.predict(X_test)


In [60]:
confusion_matrix(y_test, y_pred)


array([[123,  41],
       [ 39, 124]])

In [61]:
accuracy_score(y_test, y_pred)

0.7553516819571865

In [62]:
pos = []
neg = []
error = []
ticker2 = []
realticker =[]

In [63]:
data_test.shape[0]

48

In [80]:

for i in range(0,data_test.shape[0]):
  try:
      if clf.predict([df8['sentence'][i]])==1:
        ticker2=[data_test['ticker'][i]]
        realticker.append(ticker2)
        pos.append('1')
      else: 
        neg.append('0')
  except KeyError:
    error.append('error')
 
  

In [81]:
len(pos)

30

In [82]:
len(neg)

18

In [83]:
len(error)

0

In [84]:
realticker

[['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN']]